In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb

# **Static, partial equilibrium model**
### **Production module, example 3:**

*Example 3 defines a partial equilibrium model from data, including nesting structure, and calibration targets. The model is as stylized as possible, ignoring investments, trade and taxes.*

### **1: Read in data, and nesting structure**

Read in general equilibrium data:

In [2]:
data_folder = os.getcwd()+'\\PE_P\\Example_3'
data = {'Production_v': data_folder+'\\Production_v.xlsx', 'Production_p': data_folder+'\\Production_p.xlsx'}
components = ['domestic']
db_GE = ReadData.read_data.main(data,components=components)

Extract data on sector 's1' from GE data:

In [3]:
s0 = 's1'
db_PE = ReadData.PE_from_GE(db_GE,s0)

Read in nesting:

In [4]:
nt = nesting_tree.nesting_tree(name='test')

*Input-part:*

In [5]:
nt.add_tree(data_folder+'\\nest_s1_in.xlsx',name='s1_in')
read_type = {'1dvars': {'sheets': ['sigma'],'names':{}}, 'vars_panel': {'sheets': {'mu': 2},'names': {}}}
nt.trees['s1_in'].database.read_from_excel(data_folder+'\\nest_s1_in.xlsx',read_type)

*Output-split:*

In [6]:
nt.add_tree(data_folder+'\\nest_s1_out.xlsx',name='s1_out',**{'type_io': 'output', 'type_f': 'CET'})
nt.trees['s1_out'].database.read_from_excel(data_folder+'\\nest_s1_out.xlsx',read_type)

### **2: Set up baseline model**

*Compile tree attributes:*

In [7]:
nt.run_all()

*Initiate production module:*

In [8]:
pm = PE.GPM_STA_PE.production(nt)

In [9]:
pm.df_write(repo=os.getcwd()+'\\V1',export_settings=True)
pm.create_model_instance(name='baseline',repo=work_folder)
pm.df_run(name='baseline')

GamsExceptionExecution: GAMS return code not 0 (3), check C:\Users\sxj477\Documents\GitHub\GamsPythonModels\work_folder\_gams_py_gjo0.lst for more details